In [2]:
import pandas as pd
#from gensim.models.fasttext import FastText
import tensorflow as tf
import numpy as np

Label Read In 

In [3]:
df1 = pd.read_csv('finnum/train.csv')
x = df1.index
df2 = pd.read_csv('finnum/dev.csv')
df = pd.concat([df1,df2], ignore_index = True)
df['cat_num'] = df['category'].astype('category').cat.codes
labels = pd.get_dummies(df['cat_num'][x])
labels2 = pd.get_dummies(df['cat_num'][len(x):])

Loading input matrix

In [4]:
trainInp = np.load('trainChar.npy')
valInp = np.load('valChar.npy')
embed = np.load('embedChar.npy')
embed = embed.astype(np.float64)

In [5]:
learning_rate = 0.001
batch_size = 16

In [6]:
tf.reset_default_graph()

Batching and creating iterators

In [7]:
train = (trainInp, labels)
val = (valInp, labels2)

# create training Dataset and batch it
train_data = tf.data.Dataset.from_tensor_slices(train)
train_data = train_data.shuffle(10000) # if you want to shuffle your data
train_data = train_data.batch(batch_size)

# create validation Dataset and batch it
val_data = tf.data.Dataset.from_tensor_slices(val)
val_data = val_data.shuffle(10000) # if you want to shuffle your data
val_data = val_data.batch(batch_size)

# create one iterator and initialize it with different datasets
iterator = tf.data.Iterator.from_structure(train_data.output_types, 
                                           train_data.output_shapes)
txt, label = iterator.get_next()

train_init = iterator.make_initializer(train_data)
val_init = iterator.make_initializer(val_data)

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
embedding = tf.nn.embedding_lookup(embed, txt, partition_strategy='mod', name=None)
embedded_chars_expanded = tf.expand_dims(embedding, -1)

In [9]:
embedding

<tf.Tensor 'embedding_lookup/Identity:0' shape=(?, 547, 8) dtype=float64>

In [10]:
embedded_chars_expanded

<tf.Tensor 'ExpandDims:0' shape=(?, 547, 8, 1) dtype=float64>

Implementing looped convolution

In [11]:
pooled_outputs = []
filter_sizes = [2,3,4,5]
embedding_size = embed.shape[1]
num_filters = 64
max_length = 547
for filter_size in filter_sizes:
    filter_shape = [filter_size, embedding_size, 1, num_filters]
    W = tf.Variable(tf.truncated_normal(filter_shape, stddev=0.1), name='W')
    b = tf.Variable(tf.constant(0.1, shape=[num_filters]), name = 'b')
    conv = tf.nn.conv2d(
        embedded_chars_expanded,
        tf.cast(W,tf.float64),
        strides=[1, 1, 1, 1],
        padding='VALID',
        name='conv')
    relu = tf.nn.relu(tf.nn.bias_add(conv, tf.cast(b,tf.float64)), name="relu")
    pooled = tf.nn.max_pool(
        relu,
        ksize=[1, max_length - filter_size + 1, 1, 1],
        strides=[1, 1, 1, 1],
        padding='VALID',
        name="pool")
    pooled_outputs.append(pooled)

In [12]:
filter_shape

[5, 8, 1, 64]

In [13]:
pooled_outputs

[<tf.Tensor 'pool:0' shape=(?, 1, 1, 64) dtype=float64>,
 <tf.Tensor 'pool_1:0' shape=(?, 1, 1, 64) dtype=float64>,
 <tf.Tensor 'pool_2:0' shape=(?, 1, 1, 64) dtype=float64>,
 <tf.Tensor 'pool_3:0' shape=(?, 1, 1, 64) dtype=float64>]

Combining separate convolutional layers into 1 feed forward input

In [14]:
num_filters_total = num_filters * len(filter_sizes)
combined = tf.concat(pooled_outputs, 3)
combined_flat = tf.reshape(combined, [-1, num_filters_total])

In [15]:
combined_flat

<tf.Tensor 'Reshape:0' shape=(?, 256) dtype=float64>

Adding dense layers

In [16]:
conn = tf.layers.dense(combined_flat, 100, activation = 'relu')
conn2 = tf.layers.dense(conn, len(set(df.cat_num)))

Instructions for updating:
Use keras.layers.dense instead.


In [17]:
conn

<tf.Tensor 'dense/Relu:0' shape=(?, 100) dtype=float64>

Implementing cross entropy, loss, and optimization

In [18]:
n_epochs = 10
entropy = tf.nn.softmax_cross_entropy_with_logits(labels = label, logits = conn2)
loss = tf.reduce_mean(entropy)
optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



Prediction setup

In [19]:
preds = tf.nn.softmax(conn2)
correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(label, 1))
accuracy = tf.reduce_mean(tf.cast(correct_preds, tf.float32))

Running Neural Net

In [20]:
with tf.Session() as sess:
    #start_time = time.time()
    sess.run(tf.global_variables_initializer())
    # train the model n_epochs times

    for i in range(n_epochs): 
        
        sess.run(train_init)# drawing samples from train_data
        total_loss = 0
        total_right = 0
        n_batches = 0
        totalright = 0
        totalvalright = 0
        nvalbatches = 0
        totalvalloss = 0
        try:
            while True:
                #summary,acc,_, l = sess.run([summary_op,accuracy,optimizer, loss]) #use with scalar summary
                acc,_, l = sess.run([accuracy, optimizer, loss])                
                total_loss += l
                total_right += acc
                n_batches += 1
        except tf.errors.OutOfRangeError:
            pass
        
        sess.run(val_init)
        try:
            while True:
                accval,valloss = sess.run([accuracy, loss])
                totalvalright += accval
                nvalbatches += 1
                totalvalloss += valloss
        except tf.errors.OutOfRangeError:
            pass

        print('Average loss epoch {0}: {1}'.format(i, total_loss/n_batches))
        print('Accuracy {0}: {1}'.format(i, total_right/n_batches))    
        print('Average val loss epoch {0}: {1}'.format(i, totalvalloss/nvalbatches))
        print('Val_Accuracy {0}: {1}'.format(i, totalvalright/nvalbatches)) 
    prediction = sess.run(preds, feed_dict={txt: valInp})
    prediction = np.asarray(prediction)
   


UnknownError: Failed to get convolution algorithm. This is probably because cuDNN failed to initialize, so try looking to see if a warning log message was printed above.
	 [[node conv_3 (defined at <ipython-input-11-03109289e438>:15) ]]
	 [[node Mean (defined at <ipython-input-18-67129b422f37>:3) ]]

Caused by op 'conv_3', defined at:
  File "C:\Users\justi\Anaconda3\envs\tf_gpu\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\justi\Anaconda3\envs\tf_gpu\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\justi\Anaconda3\envs\tf_gpu\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\justi\Anaconda3\envs\tf_gpu\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\justi\Anaconda3\envs\tf_gpu\lib\site-packages\ipykernel\kernelapp.py", line 505, in start
    self.io_loop.start()
  File "C:\Users\justi

Calculating accuracy for train

In [73]:
np.mean(np.equal(np.argmax(prediction, 1), labels2.idxmax(axis = 1)))

0.7728494623655914

Fooling around with numpy

In [44]:
f = np.array([[[1,2,3], [1,2,3]], [[1,2,3], [1,2,3]], [[1,2,3], [1,2,3]]])
#f = np.array([[1,2,3], [1,2,3]])
#print(f)
g = np.expand_dims(f, axis = 1)
g = np.reshape(f, [-1,2])
print(g)
print(g.shape)
print(np.squeeze(g))

[[1 2]
 [3 1]
 [2 3]
 [1 2]
 [3 1]
 [2 3]
 [1 2]
 [3 1]
 [2 3]]
(9, 2)
[[1 2]
 [3 1]
 [2 3]
 [1 2]
 [3 1]
 [2 3]
 [1 2]
 [3 1]
 [2 3]]
